<a href="https://colab.research.google.com/github/willstauffernorris/DS-Unit-1-Sprint-2-Statistics/blob/master/DS-Unit-1-Sprint-2-Statistics/Copy_of_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

1) Load and clean the data (or determine the best method to drop observations when running tests)

In [27]:
### YOUR CODE STARTS HERE

import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',
                 header=None,
                 na_values="?",
                 names=['party', 'handicapped-infants', 'water-project', 
'budget', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups', 
'anti-satellite-ban', 'aid-to-contras', 'mx-missile', 'immigration', 
'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 
'south-africa'])

df.head()




,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [28]:
df = df.replace({'y':1, 'n':0})

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [29]:
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [0]:
### a worse way to filter than the next code block below

#dem_filter = (df['party']=='democrat')

#dem = df[dem_filter]

#print(dem['budget'].head(10))


#rep_filter = (df['party']=='republican')

#rep = df[rep_filter]

#rep['budget'].head(10)





In [30]:
rep = df[df['party']=='republican']
print(rep.head(10))

dem = df[df['party']=='democrat']
dem.head(10)

         party  handicapped-infants  ...  duty-free  south-africa
0   republican                  0.0  ...        0.0           1.0
1   republican                  0.0  ...        0.0           NaN
7   republican                  0.0  ...        NaN           1.0
8   republican                  0.0  ...        0.0           1.0
10  republican                  0.0  ...        0.0           0.0
11  republican                  0.0  ...        NaN           NaN
14  republican                  0.0  ...        0.0           NaN
15  republican                  0.0  ...        0.0           NaN
18  republican                  0.0  ...        0.0           0.0
28  republican                  1.0  ...        0.0           1.0

[10 rows x 17 columns]


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0
9,democrat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
12,democrat,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN
13,democrat,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,0.0,0.0,1.0,NaN
16,democrat,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,1.0,1.0,1.0,NaN,0.0,0.0,1.0
17,democrat,1.0,NaN,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [32]:
print(rep.isnull().sum())
dem.isnull().sum()

party                    0
handicapped-infants      3
water-project           20
budget                   4
physician-fee-freeze     3
el-salvador-aid          3
religious-groups         2
anti-satellite-ban       6
aid-to-contras          11
mx-missile               3
immigration              3
synfuels                 9
education               13
right-to-sue            10
crime                    7
duty-free               12
south-africa            22
dtype: int64


party                    0
handicapped-infants      9
water-project           28
budget                   7
physician-fee-freeze     8
el-salvador-aid         12
religious-groups         9
anti-satellite-ban       8
aid-to-contras           4
mx-missile              19
immigration              4
synfuels                12
education               18
right-to-sue            15
crime                   10
duty-free               16
south-africa            82
dtype: int64

** 2) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01 **

In [34]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

df['budget'].head()

0    0.0
1    0.0
2    1.0
3    1.0
4    1.0
Name: budget, dtype: float64

In [37]:

stats.ttest_ind(dem['budget'], rep['budget'], nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [41]:
print(dem['budget'].mean())
print(rep['budget'].mean())

0.8884615384615384
0.13414634146341464


On the 'budget' issue, the p value is almost zero and my t statistic is ~23. This means that I reject the null hypothesis that Republicans and Democrats are voting the same on the 'budget' issue, because my p value is <.05. There is a significant difference between the parties on this issue.

When I print the means of each parties' vote, it's clear that the Democrats overwhelming favor the 'budget' bill compared to Republicans (88% of Dems vs 13% of GOP).


**3) Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01**

In [44]:
##last question, I guessed at random based on what I know about politics and 
#found a significant difference on an issue. For this quesetion, I will pick 
#a topic and explore the means before I do a t test.

##more democratitc
print(dem['anti-satellite-ban'].mean())
print(rep['anti-satellite-ban'].mean())

## more republican
print(dem['education'].mean())
print(rep['education'].mean())

stats.ttest_ind(dem['education'], rep['education'], nan_policy = 'omit')

0.7722007722007722
0.24074074074074073
0.14457831325301204
0.8709677419354839


Ttest_indResult(statistic=-20.500685724563073, pvalue=1.8834203990450192e-64)

On the 'education' bill, I reject the null hypothesis that the GOP and Dems vote the same due to the p<.05. Republicans significantly favor the bill.


** 4) Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference) **


In [47]:
print(dem.describe())
print(rep.describe())
##water project looks like it garnered similar levels of support

stats.ttest_ind(dem['water-project'], rep['water-project'], nan_policy = 'omit')

##p value is >.05, so there is not a statistically different vote between Democrats and Republicans on the water project bill.

       handicapped-infants  water-project  ...   duty-free  south-africa
count           258.000000     239.000000  ...  251.000000    185.000000
mean              0.604651       0.502092  ...    0.637450      0.935135
std               0.489876       0.501045  ...    0.481697      0.246956
min               0.000000       0.000000  ...    0.000000      0.000000
25%               0.000000       0.000000  ...    0.000000      1.000000
50%               1.000000       1.000000  ...    1.000000      1.000000
75%               1.000000       1.000000  ...    1.000000      1.000000
max               1.000000       1.000000  ...    1.000000      1.000000

[8 rows x 16 columns]
       handicapped-infants  water-project  ...   duty-free  south-africa
count           165.000000     148.000000  ...  156.000000    146.000000
mean              0.187879       0.506757  ...    0.089744      0.657534
std               0.391804       0.501652  ...    0.286735      0.476168
min               0.000000  

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)